In [1]:
import sys
sys.path.append("..")

import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from ipywidgets import interact

# from conc_obj import EEGData
from eegdata_multi import EEGData
from utils.plt import plot_psd, plot_montage
from utils.ica import plot_ica_comp

# MNE imports
import mne
from mne.io.edf import read_raw_edf
from mne.datasets import eegbci
# from mne.decoding import CSP
from csp.CSPObj import CSP

from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA, FastICA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

import json


***Macros***

>General use macros, importing JSON files to use as the configuration files

In [2]:
script_path = Path().resolve()
main_folder = (script_path / "../").resolve()
proj_folder = (script_path / "../../").resolve()

JSON_MAIN_PATH = main_folder / "config/config_main.json"
JSON_CSP_PATH = main_folder / "config/config_csp.json"
JSON_GRID_PATH = main_folder / "config/config_grid.json"
EVENTS_PATH = main_folder / "config/events.json"

with open(JSON_MAIN_PATH, "r") as f:
    config_main = json.load(f)

with open(JSON_CSP_PATH, "r") as f:
    config_csp = json.load(f)
    
with open(JSON_GRID_PATH, "r") as f:
    json_grid = json.load(f)

VERBOSE = config_main['verbose'].lower() == 'true'

L_FREQ = config_main['l_freq']
H_FREQ = config_main['h_freq']

N_SUBJECTS = config_main["n_subjects"]
N_COMPONENTS_ICA = config_main["n_components_ica"]
N_COMPONENTS_CSP = config_csp["n_components"]
N_COMPONENTS_PCA = N_COMPONENTS_CSP

"""
T0 corresponds to rest
T1 corresponds to onset of motion (real or imagined) of
the left fist (in runs 3, 4, 7, 8, 11, and 12)
both fists (in runs 5, 6, 9, 10, 13, and 14)
T2 corresponds to onset of motion (real or imagined) of
the right fist (in runs 3, 4, 7, 8, 11, and 12)
both feet (in runs 5, 6, 9, 10, 13, and 14)
"""


'\nT0 corresponds to rest\nT1 corresponds to onset of motion (real or imagined) of\nthe left fist (in runs 3, 4, 7, 8, 11, and 12)\nboth fists (in runs 5, 6, 9, 10, 13, and 14)\nT2 corresponds to onset of motion (real or imagined) of\nthe right fist (in runs 3, 4, 7, 8, 11, and 12)\nboth feet (in runs 5, 6, 9, 10, 13, and 14)\n'

***Initialization of EEG object***

>***(If the files are not locally stored, it will download them to the user system automatically)***

>***Use of functions like .filter_data() also is obligatory if there is no data stored locally***

In [3]:
eeg_obj = EEGData(config_main, config_csp, proj_folder, verbose=VERBOSE)

#* Filters data and plots PSD to see differences
# eeg_obj.filter_data()
# eeg_obj.plot_psd_ba_filt(verbose=VERBOSE)

# eeg_obj.plot_psd(verbose=VERBOSE)

#* Normalizes data
# eeg_obj.normalize_data()

Loaded data:
<Raw | sample_mmi_h_raw.fif, 64 x 2333120 (14582.0 s), ~1.11 GiB, data loaded> <Raw | sample_mmi_hf_raw.fif, 64 x 2333120 (14582.0 s), ~1.11 GiB, data loaded>
<Raw | sample_mmi_h_filt_raw.fif, 64 x 2333120 (14582.0 s), ~1.11 GiB, data loaded> <Raw | sample_mmi_hf_filt_raw.fif, 64 x 2333120 (14582.0 s), ~1.11 GiB, data loaded>
Reading /Users/Shared/42/ML/projects/total_perspective_vortex/data/ica/sample_mmi_h_ica.fif ...
Now restoring ICA solution ...
Ready.
Reading /Users/Shared/42/ML/projects/total_perspective_vortex/data/ica/sample_mmi_hf_ica.fif ...
Now restoring ICA solution ...
Ready.
<ICA | raw data decomposition, method: fastica (fit in 85 iterations on 2333120 samples), 64 ICA components (64 PCA components available), channel types: eeg, 3 sources marked for exclusion> <ICA | raw data decomposition, method: fastica (fit in 80 iterations on 2333120 samples), 64 ICA components (64 PCA components available), channel types: eeg, 3 sources marked for exclusion>
<ICA | r

***Prediction - Loading testing data & ML models***

>***If we don't want to train our model and want to make predictions or see any metric right away, we can load the data and call the methods right away.***

>***Take into account that in order to change the events or any other parameter in the tools/methods loaded, you will have to train and save the entire model/data again!***

In [4]:

# X_test, y_test = eeg_obj.load_models()
# eeg_obj.pred(X_test, y_test)

**Basic information and montage plotting in 2D & 3D**
> ***The channel names can also be printed***

In [5]:

#* Plots different montages in 2D & 3D
# data = eeg_obj.get_raw_h()
# ch_names = data.info["ch_names"] 
# plot_montage(eeg_obj.montage, ch_names)

**ICA(Independent Component Analysys)**
> ***The number of components that ICA will try to sort out can be changed, it is advised to use values in the range [15-45]***

> ***Ocular artifacts are also removed, since they don't contribute to the muscular movement on this evaluation***

> ***The components can also be plotted and ocular artifacts, EOG, will be clearly visible***

In [6]:

#* Computes ICA components
# eeg_obj.decomp_ica(n_components=N_COMPONENTS_ICA, plt_show=True, verbose=VERBOSE)

#* Plot components of ICA
# plot_ica_comp(folder / config["path_ica_h"])

**Specify events & create Epochs**
> ***The events used along with the JSON configuration will be crucial for the ML algorimths to work properly***


In [7]:
data, _ = eeg_obj.get_clean()
events, _ = eeg_obj.get_events()

event_l = config_csp["ev_mlist_eight"]
groupeve_dict = config_csp["event_dict_hf"]

event_dict1 = {key: value for key, value in groupeve_dict.items() if value in event_l[0]}
print("Event dict. :", event_dict1)

print()
epochs = mne.Epochs(data, events, event_id=event_dict1, tmin=0.3, tmax=3.3, baseline=None, verbose=VERBOSE)
data = epochs.get_data()
# data = data.reshape(data.shape[0], -1)

labels = epochs.events[:, -1]
print("X:", data.shape)
print("Y:", labels.shape)

Event dict. : {'do/feet': 1, 'do/hands': 2, 'imagine/feet': 3, 'imagine/hands': 4, 'rest': 5}

Using data from preloaded Raw for 3540 events and 481 original time points ...
0 bad epochs dropped
X: (3540, 64, 481)
Y: (3540,)


***Pipeline***

>The dimensionality reduction tools, classifications algorimths and signal processing (CSP) are included.

>The default values/functions have been proved to be good over tests.

In [8]:
svm_clf = SVC(kernel='rbf', C=100, gamma=2, probability=True)
rf_clf = RandomForestClassifier(n_estimators=200, max_depth=None, random_state=42)

ensemble = VotingClassifier(estimators=[('svm', svm_clf), ('rf', rf_clf)], voting='soft')

# Create a new pipeline for LDA
pipeline = Pipeline([
    #* Does not work (it needs reshaping)
    # ('ica', FastICA(n_components=64)),

    ('csp', CSP(n_components=N_COMPONENTS_CSP, reg='ledoit_wolf', log=True, norm_trace=False)),

    ('scaler', StandardScaler()), #* StandardScaler works best
    # ('scaler', MinMaxScaler()),
    # ('scaler', RobustScaler()),

    ('pca', PCA(n_components=N_COMPONENTS_PCA)),

	('voting_cs', ensemble)
    #* Choose only one of the following classifiers
	# ('lda', LDA())
    # ('svm', SVC(kernel='rbf', C=100, gamma=2))
	# ('rf', RandomForestClassifier())
])

# Fit the 3D pipeline to the transformed data
# pipeline.fit(data, labels)


***GridSearch - Parameter selection***

>Exhaustive search over specified parameter values for an estimator.

>The default values have been tested. Performance varies from event type selection.

In [9]:
from utils.gridsearch import grid_finder, grid_search

# grid = grid_finder(json_grid, 'svm', 'wide')
# print(grid)
# grid_search(data, labels, pipeline, grid)

***K-Fold Training/Testing***

>Split the data to train over different data sets, improves generalization.

>Remove the last step of the pipeline to get the proccesed data without the ML CLF algorimth

In [10]:

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Fit the pipeline to the data
pipeline.fit(data, labels)

# Transform the data using the pipeline
# Create a new pipeline excluding the last step
pipeline_without_last_step = Pipeline(pipeline.steps[:-1])

# Transform the data using the new pipeline
processed_data = pipeline_without_last_step.transform(data)
print(processed_data.shape)


(3540, 12)


***Cross Validation - The good ol' tester***

>We feed it the n_splits we chose on the K-folds along with the pipeline.

>It ensures that the training/testing datasets are not mixed & calculates the average score over the K-folds.

In [11]:
# Perform cross-validation
scores = cross_val_score(pipeline, data, labels, cv=cv)

print("Cross-validation scores:", scores)
print("Processed data shape:", processed_data.shape)

Cross-validation scores: [0.89548023 0.87429379 0.88559322 0.87288136 0.90677966]
Processed data shape: (3540, 12)


In [12]:

#* Divide the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(processed_data, labels, test_size=0.2, random_state=42)

eeg_obj.train_model(X_train, y_train)

eeg_obj.pred(X_test, y_test, n_preds=30)

Train Accuracy LDA: 0.5709745762711864
Train Accuracy SVM: 1.0
Train Accuracy RF: 1.0

epoch nb: [prediction] [truth] equal?
epoch 000:	[5]		[5]  True
epoch 001:	[1]		[1]  True
epoch 002:	[5]		[5]  True
epoch 003:	[5]		[5]  True
epoch 004:	[5]		[5]  True
epoch 005:	[5]		[5]  True
epoch 006:	[5]		[5]  True
epoch 007:	[5]		[5]  True
epoch 008:	[5]		[5]  True
epoch 009:	[1]		[1]  True
epoch 010:	[4]		[4]  True
epoch 011:	[5]		[5]  True
epoch 012:	[3]		[3]  True
epoch 013:	[5]		[5]  True
epoch 014:	[5]		[5]  True
epoch 015:	[3]		[3]  True
epoch 016:	[1]		[1]  True
epoch 017:	[5]		[5]  True
epoch 018:	[5]		[5]  True
epoch 019:	[5]		[5]  True
epoch 020:	[3]		[3]  True
epoch 021:	[2]		[2]  True
epoch 022:	[5]		[5]  True
epoch 023:	[5]		[5]  True
epoch 024:	[3]		[3]  True
epoch 025:	[5]		[3]  False
epoch 026:	[5]		[5]  True
epoch 027:	[5]		[5]  True
epoch 028:	[5]		[5]  True
epoch 029:	[5]		[5]  True
epoch 030:	[1]		[1]  True

LDA Accuracy: 0.5932203389830508
SVM Accuracy: 0.8983050847457628
R

**Saving data locally**
> ***Saving and loading data locally saves time and computational power***

> ***Note that it is also needed to change the configuration at the JSON files to import local files***

In [13]:
#* Saves filtered and concatenated data for faster loading
# eeg_obj.save_type_data(type="raw")
# eeg_obj.save_type_data(type="filtered")
# eeg_obj.save_type_data(type="norm")
# eeg_obj.save_type_data(type="ica")
# eeg_obj.save_models()